In [10]:
# ======================================
# Data Centres + Substations + Overhead Lines
# with Voltage Toggle + Legend
# ======================================

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pyproj import Transformer
import folium

# --- 1. Load substations (CSV) ---
subs = pd.read_csv("/content/gb_substations_data_281118.csv")

# Convert OSGB36 (EASTING/NORTHING) → WGS84 (lon/lat)
transformer = Transformer.from_crs("epsg:27700", "epsg:4326", always_xy=True)
subs["lon"], subs["lat"] = transformer.transform(subs["EASTING"].values, subs["NORTHING"].values)

gdf_subs = gpd.GeoDataFrame(
    subs,
    geometry=gpd.points_from_xy(subs.lon, subs.lat),
    crs="EPSG:4326"
)

# --- 2. Load data centres ---
data = pd.read_csv("/content/combined_postcodes_geocoded.csv")
gdf_data = gpd.GeoDataFrame(
    data,
    geometry=gpd.points_from_xy(data.lon, data.lat),
    crs="EPSG:4326"
)

# --- 3. Load Overhead Lines shapefile ---
gdf_ohl = gpd.read_file("/content/OHL.shp")
if gdf_ohl.crs is None:
    print("⚠️ No CRS found for OHL.shp — assuming EPSG:27700 (British National Grid)")
    gdf_ohl.set_crs(epsg=27700, inplace=True)
gdf_ohl = gdf_ohl.to_crs(epsg=4326)

# --- 4. Setup folium map ---
m = folium.Map(location=[54.5, -3], zoom_start=6, tiles="cartodb positron")

# Define colour by voltage
def voltage_colour(v):
    if v >= 400:
        return "purple"
    elif v >= 275:
        return "red"
    elif v >= 132:
        return "orange"
    else:
        return "gray"

# --- 5. Create FeatureGroups for toggling ---
layer_400 = folium.FeatureGroup(name="400 kV Substations", show=True)
layer_275 = folium.FeatureGroup(name="275 kV Substations", show=True)
layer_132 = folium.FeatureGroup(name="132 kV Substations", show=True)
layer_data = folium.FeatureGroup(name="Data Centres", show=True)
layer_ohl = folium.FeatureGroup(name="Overhead Lines", show=False)

# --- 6. Add substations by voltage ---
for _, row in gdf_subs.iterrows():
    v = row["VOLTAGE_HIGH"]
    if pd.isna(v):
        continue
    colour = voltage_colour(v)
    marker = folium.CircleMarker(
        location=[row.lat, row.lon],
        radius=4,
        color=colour,
        fill=True,
        fill_color=colour,
        fill_opacity=0.8,
        popup=(f"<b>{row['SUBST_NAME']}</b><br>{v} kV<br>{row['ASSET_TYPE']}")
    )
    if v >= 400:
        marker.add_to(layer_400)
    elif v >= 275:
        marker.add_to(layer_275)
    elif v >= 132:
        marker.add_to(layer_132)

# --- 7. Add data centres (uniform colour) ---
for _, row in gdf_data.iterrows():
    folium.CircleMarker(
        location=[row.lat, row.lon],
        radius=4,
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.8,
        popup=f"<b>Data Centre:</b> {row['postcode']}"
    ).add_to(layer_data)

# --- 8. Add Overhead Lines ---
folium.GeoJson(
    gdf_ohl,
    name="Overhead Lines",
    style_function=lambda x: {"color": "green", "weight": 1, "opacity": 0.6}
).add_to(layer_ohl)

# --- 9. Add all layers to map ---
layer_400.add_to(m)
layer_275.add_to(m)
layer_132.add_to(m)
layer_data.add_to(m)
layer_ohl.add_to(m)

# --- 10. Add legend ---
legend_html = """
<div style="
position: fixed;
bottom: 20px; left: 20px; width: 160px; height: 160px;
background-color: white; z-index:9999; font-size:14px;
border:2px solid grey; border-radius:10px; padding:10px;">
<b>Voltage Level</b><br>
<i style='color:purple'>⬤</i> 400 kV<br>
<i style='color:red'>⬤</i> 275 kV<br>
<i style='color:orange'>⬤</i> 132 kV<br>
<i style='color:blue'>⬤</i> Data Centres<br>
<i style='color:green'>━</i> Overhead Lines
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# --- 11. Layer control + save ---
folium.LayerControl(collapsed=False).add_to(m)
m.save("data_centre_infrastructure_map.html")
print("✅ Map saved as data_centre_infrastructure_map.html")
m


Output hidden; open in https://colab.research.google.com to view.